In [1]:
print("ok")

ok


In [2]:
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px

import os
import os
import sys
import pandas as pd
from typing import List, Dict, Optional
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import warnings

warnings.filterwarnings("ignore")
import plotly.express as px

In [3]:
dataset_path = r'C:\Users\TPWODL\New folder_Content\AutonomousDataAnalystAgent\data\raw\x_data.xlsx'

In [4]:
df = pd.read_excel(r'C:\Users\TPWODL\New folder_Content\AutonomousDataAnalystAgent\data\raw\x_data.xlsx')


In [5]:
df.shape

(32873, 34)

In [6]:
df.head(1)

,SL.NO,DATE,SHIFT DUTY,QUERY/REQUEST/COMPLAINT,COMPLAINT DETAILS,COMPLAINT NUMBER,SECTION,SUB-DIVISION,DIVISION,CIRCLE,...,ARREARS,REPEAT (Y / N),FORWARDED TO,SENTIMENTS,NATURE OF TWEET,ACTIONABLE/ NON ACTIONABLE,AGEING,SLAB,SLAB2,MINUTE
0,1,2022-06-10,NaN,NaN,Mr. Mohan Maharana complaint regarding the fun...,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,1303.0,>180,>365,0.0


In [7]:
df_open = df[df['CLOSED/OPEN'] == 'open']

In [8]:
pivot = pd.pivot_table(
    df_open,
    values='CLOSED/OPEN',          # numeric or countable field
    index='COMPLAINT TYPE',        # rows
    columns='DEPT',                # columns
    aggfunc='count',               # count complaints by dept
    fill_value=0
)
# ➡️ Add Grand Total column (row-wise sum)
pivot['Grand_Total'] = pivot.sum(axis=1)

# ➡️ Add Grand Total row (column-wise sum)
pivot.loc['Grand_Total'] = pivot.sum(axis=0)

In [9]:
pivot

DEPT,Grand_Total
COMPLAINT TYPE,
Grand_Total,0.0


In [10]:
import pandas as pd

def generate_complaint_pivot(dataset_path: str) -> pd.DataFrame:
    """
    Generate a pivot table of open complaints by department and complaint type,
    including grand totals for rows and columns.

    Parameters
    ----------
    datapath : str
        Path to the CSV file containing complaint data.

    Returns
    -------
    pd.DataFrame
        Pivot table with counts of open complaints, plus grand totals.
    """
    # Load dataset
    df = pd.read_excel(dataset_path)

    # Filter only open complaints
    df_open = df[df['CLOSED/OPEN'].str.lower() == 'open']
    
    # Create pivot table
    pivot = pd.pivot_table(
        df_open,
        values='CLOSED/OPEN',
        index='COMPLAINT TYPE',
        columns='DEPT',
        aggfunc='count',
        fill_value=0
    )

    # Add Grand Total column (row-wise sum)
    pivot['Grand_Total'] = pivot.sum(axis=1)

    # Add Grand Total row (column-wise sum)
    pivot.loc['Grand_Total'] = pivot.sum(axis=0)

    return pivot


In [11]:
pvt = generate_complaint_pivot(dataset_path)

In [12]:
pvt

DEPT,Commercial,O&M,Other,Grand_Total
COMPLAINT TYPE,,,,
Billing,1,0,0,1
Civil works,0,148,0,148
Low Voltage,0,1,0,1
NSC,8,0,0,8
Others,0,0,12,12
Pole Shifting / Lt Sagging,0,72,0,72
Safety,0,66,0,66
Solar,1,0,0,1
Transformer failure / NCC,0,2,0,2


In [13]:
def open_complaint_pivot(dataset_path: str) -> pd.DataFrame:
    """
    Generate a pivot table of open complaints by department and complaint type,
    including grand totals for rows and columns.

    Parameters
    ----------
    dataset_path : str
        Path to the Excel file containing complaint data.

    Returns
    -------
    pd.DataFrame
        Pivot table with counts of open complaints, plus grand totals.
    """
    # Load dataset
    df = pd.read_excel(dataset_path)

    # Filter only open complaints (handle case-insensitive and NaN values)
    df_open = df[df['CLOSED/OPEN'].str.lower().str.strip() == 'open']
    
    # Create pivot table
    pivot = pd.pivot_table(
        df_open,
        values='CLOSED/OPEN',
        index='COMPLAINT TYPE',
        columns='DEPT',
        aggfunc='count',
        fill_value=0
    )

    # Add Grand Total column (row-wise sum)
    pivot['Grand_Total'] = pivot.sum(axis=1)

    # Add Grand Total row (column-wise sum)
    pivot.loc['Grand_Total'] = pivot.sum(axis=0)

    # **FIX: Convert all numpy types to native Python types**
    pivot = pivot.astype(int)  # Convert to Python int
    
    # Reset index to make 'COMPLAINT TYPE' a regular column
    pivot = pivot.reset_index()

    return pivot

In [14]:
vt = open_complaint_pivot(dataset_path)

In [15]:
vt

DEPT,COMPLAINT TYPE,Commercial,O&M,Other,Grand_Total
0,Billing,1,0,0,1
1,Civil works,0,148,0,148
2,Low Voltage,0,1,0,1
3,NSC,8,0,0,8
4,Others,0,0,12,12
5,Pole Shifting / Lt Sagging,0,72,0,72
6,Safety,0,66,0,66
7,Solar,1,0,0,1
8,Transformer failure / NCC,0,2,0,2
9,safety,0,1,0,1


In [ ]:
def open_close_complaint_pivot(dataset_path: str) -> pd.DataFrame:
    """
    Reads an Excel dataset and returns a pivot table of complaints
    grouped by COMPLAINT TYPE, Department, and Open/Closed status,
    with grand totals added.
    
    Parameters
    ----------
    dataset_path : str
        Path to the Excel file containing complaint data.
    
    Returns
    -------
    pd.DataFrame
        Pivot table with flattened columns and grand totals.
    """
    # Load dataset
    df = pd.read_excel(dataset_path)
    
    # **FIX: Clean data before pivoting**
    # Remove any whitespace and handle case sensitivity
    df['COMPLAINT TYPE'] = df['COMPLAINT TYPE'].astype(str).str.strip()
    df['DEPT'] = df['DEPT'].astype(str).str.strip()
    df['CLOSED/OPEN'] = df['CLOSED/OPEN'].astype(str).str.strip().str.title()  # Capitalize properly

    # Build pivot table
    pivot_data = pd.pivot_table(
        df,
        index='COMPLAINT TYPE',
        columns=['DEPT', 'CLOSED/OPEN'],
        aggfunc='size',
        fill_value=0
    )

    # **FIX: Flatten multi-level columns before adding totals**
    # This creates column names like "DEPT_STATUS" instead of tuples
    pivot_data.columns = ['_'.join(map(str, col)).strip('_') for col in pivot_data.columns]

    # Add Grand Total column (row-wise sum)
    pivot_data['Grand_Total'] = pivot_data.sum(axis=1)

    # Add Grand Total row (column-wise sum)
    pivot_data.loc['Grand_Total'] = pivot_data.sum(numeric_only=True, axis=0)

    # Ensure integer values (convert from float to int)
    pivot_data = pivot_data.astype(int)

    # Reset index to make 'COMPLAINT TYPE' a regular column
    pivot_data = pivot_data.reset_index()
    dict_pivot_data= pivot_data.to_dict()

    return dict_pivot_data


In [23]:
dict_pivot = open_close_complaint_pivot(dataset_path)

In [26]:
print(dict_pivot.keys())

dict_keys(['COMPLAINT TYPE', 'Commercial_Closed', 'Commercial_Open', 'O&M_Closed', 'O&M_Open', 'Other_Closed', 'Other_Open', 'Grand_Total'])


In [27]:
for key, value in dict_pivot.items(): print(key, ":", value)

COMPLAINT TYPE : {0: 'Billing', 1: 'Civil Works', 2: 'Civil works', 3: 'Electrification', 4: 'Low Voltage', 5: 'Metering', 6: 'NO Power Supply', 7: 'NSC', 8: 'No Power Supply', 9: 'Others', 10: 'Payment', 11: 'Pole Shifting / Lt Sagging', 12: 'Portal Problem', 13: 'Power Outage', 14: 'Safety', 15: 'Solar', 16: 'Transformer failure / NCC', 17: 'billing', 18: 'nsc', 19: 'others', 20: 'safety', 21: 'Grand_Total'}
Commercial_Closed : {0: 818, 1: 0, 2: 0, 3: 0, 4: 0, 5: 193, 6: 0, 7: 768, 8: 0, 9: 0, 10: 140, 11: 0, 12: 253, 13: 0, 14: 0, 15: 72, 16: 0, 17: 1, 18: 1, 19: 0, 20: 0, 21: 2246}
Commercial_Open : {0: 1, 1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: 8, 8: 0, 9: 0, 10: 0, 11: 0, 12: 0, 13: 0, 14: 0, 15: 1, 16: 0, 17: 0, 18: 0, 19: 0, 20: 0, 21: 10}
O&M_Closed : {0: 0, 1: 1, 2: 1752, 3: 643, 4: 1311, 5: 0, 6: 8, 7: 0, 8: 6686, 9: 381, 10: 0, 11: 972, 12: 0, 13: 11444, 14: 1783, 15: 0, 16: 794, 17: 0, 18: 0, 19: 0, 20: 44, 21: 25819}
O&M_Open : {0: 0, 1: 0, 2: 148, 3: 0, 4: 1, 5: 0, 6: 0, 

In [ ]:
import pandas as pd

def agging_open_close_pivot_dict(dataset_path: str) -> dict:
    """
    Reads an Excel file, creates a pivot table of complaint type vs age bucket,
    adds grand totals (row + column), and returns the result as a dictionary.
    
    Parameters
    ----------
    dataset_path : str
        Path to the Excel dataset.
    
    Returns
    -------
    dict
        Dictionary representation of the pivot table with totals.
    """
    # Load dataset
    old_df = pd.read_excel(dataset_path)
    df = old_df.copy()

    # Ensure DATE column is datetime
    df['DATE'] = pd.to_datetime(df['DATE'])

    # Calculate age in days
    today = pd.Timestamp.today()
    df['Age_Days'] = (today - df['DATE']).dt.days

    # Define age buckets (fixed bin ranges)
    bins = [0, 15, 30, 60, 90, 180, float('inf')]
    labels = ['<15Days', '16-30Days', '31-60Days', '61-90Days', '91-180Days', '>180Days']
    df['Age_Bucket'] = pd.cut(df['Age_Days'], bins=bins, labels=labels, right=True, include_lowest=True)    
    
    # Pivot table with complaint type + age bucket
    pivot_data = pd.pivot_table(
        df,
        values='CLOSED/OPEN',           # column to count
        index='COMPLAINT TYPE',          # rows (no list needed for single column)
        columns='Age_Bucket',            # columns (no list needed for single column)
        aggfunc='count',
        fill_value=0
    )
    
    # Reorder columns to match the age bucket order
    pivot_data = pivot_data[labels]
    
    # Add Grand Total column (row-wise sum)
    pivot_data['Grand_Total'] = pivot_data.sum(axis=1)
    
    # Add Grand Total row (column-wise sum)
    pivot_data.loc['Grand_Total'] = pivot_data.sum(axis=0)
    
    # Ensure integers
    pivot_data = pivot_data.astype(int)
    
    # Reset index to make 'COMPLAINT TYPE' a regular column
    pivot_data = pivot_data.reset_index()
    
    # Convert to dictionary (orient='records' for better format)
    dict_pivot_data = pivot_data.to_dict(orient='records')
    
    return dict_pivot_data

In [ ]:
aging = agging_open_close_pivot_dict(dataset_path)

In [32]:
aging_df =pd.DataFrame(aging)

In [33]:
aging_df

,COMPLAINT TYPE,<15Days,16-30Days,31-60Days,61-90Days,91-180Days,>180Days,Grand_Total
0,Billing,1,17,23,20,55,703,819
1,Civil Works,0,0,0,0,0,1,1
2,Civil works,37,30,38,46,233,1516,1900
3,Electrification,0,0,0,0,0,643,643
4,Low Voltage,7,4,28,26,161,1086,1312
5,Metering,0,0,3,0,8,182,193
6,NO Power Supply,0,0,0,0,0,8,8
7,NSC,23,24,30,17,66,616,776
8,No Power Supply,26,61,105,214,1228,5052,6686
9,Others,35,49,116,126,539,3951,4816


In [36]:
import pandas as pd

def agging_open_pivot_dict(dataset_path: str) -> dict:
    """
    Reads an Excel file, creates a pivot table of complaint type vs age bucket,
    adds grand totals (row + column), and returns the result as a dictionary.
    
    Parameters
    ----------
    dataset_path : str
        Path to the Excel dataset.
    
    Returns
    -------
    dict
        Dictionary representation of the pivot table with totals.
    """
    # Load dataset
    old_df = pd.read_excel(dataset_path)
    
    # Filter for open complaints (FIXED: was referencing 'df' instead of 'old_df')
    df = old_df[old_df['CLOSED/OPEN'].str.lower().str.strip() == 'open']
    
    # Ensure DATE column is datetime
    df['DATE'] = pd.to_datetime(df['DATE'])

    # Calculate age in days
    today = pd.Timestamp.today()
    df['Age_Days'] = (today - df['DATE']).dt.days

    # Define age buckets (FIXED: bin ranges to match labels correctly)
    bins = [0, 15, 30, 60, 90, 180, float('inf')]
    labels = ['<15Days', '16-30Days', '31-60Days', '61-90Days', '91-180Days', '>180Days']
    df['Age_Bucket'] = pd.cut(df['Age_Days'], bins=bins, labels=labels, right=True, include_lowest=True)    
    
    # Pivot table with complaint type + age bucket
    pivot_data = pd.pivot_table(
        df,
        values='CLOSED/OPEN',           # column to count
        index='COMPLAINT TYPE',          # rows (no list needed for single column)
        columns='Age_Bucket',            # columns (no list needed for single column)
        aggfunc='count',
        fill_value=0
    )
    
    # Reorder columns to match the age bucket order
    pivot_data = pivot_data[labels]
    
    # Add Grand Total column (row-wise sum)
    pivot_data['Grand_Total'] = pivot_data.sum(axis=1)
    
    # Add Grand Total row (column-wise sum)
    pivot_data.loc['Grand_Total'] = pivot_data.sum(axis=0)
    
    # Ensure integers
    pivot_data = pivot_data.astype(int)
    
    # Reset index to make 'COMPLAINT TYPE' a regular column
    pivot_data = pivot_data.reset_index()
    
    # Convert to dictionary (orient='records' for better format)
    dict_pivot_data = pivot_data.to_dict(orient='records')
    
    return dict_pivot_data

In [37]:
aging_open = agging_open_pivot_dict(dataset_path)

In [39]:
aging_open = pd.DataFrame(aging_open)

In [40]:
aging_open

,COMPLAINT TYPE,<15Days,16-30Days,31-60Days,61-90Days,91-180Days,>180Days,Grand_Total
0,Billing,0,0,1,0,0,0,1
1,Civil works,6,11,17,15,53,46,148
2,Low Voltage,0,0,1,0,0,0,1
3,NSC,5,3,0,0,0,0,8
4,Others,1,4,5,1,1,0,12
5,Pole Shifting / Lt Sagging,1,4,5,5,26,31,72
6,Safety,4,1,6,10,21,24,66
7,Solar,1,0,0,0,0,0,1
8,Transformer failure / NCC,2,0,0,0,0,0,2
9,safety,0,0,0,0,0,1,1
